In [ ]:
pip install sqlite3

In [ ]:
import sqlite3
com = sqlite3.connect('todo.db')
com.execute("CREATE TABLE todo (id INTEGER PRIMARY KEY, task char(100) NOT NULL,status bool NOT NULL)")
com.execute("INSERT INTO todo (task,status) VALUES('Crear hola mundo en bottle',1)")
com.execute("INSERT INTO todo (task,status) VALUES('Documentar su creacion en jupyter',1)")
com.execute("INSERT INTO todo (task,status) VALUES('Crear un proyecto nuevo de una lista ToDo',1)")
com.execute("INSERT INTO todo (task,status) VALUES('Documentar creacion de ToDo en jupyter',0)")
com.commit()

In [ ]:
import sqlite3
from bottle import Bottle, run

app = Bottle()

@app.route('/todo')
@app.route('/')
def todo_list():
    connection = sqlite3.connect('todo.db')
    cursor = connection.cursor()
    cursor.execute("SELECT id, task FROM todo WHERE status LIKE '1'")
    result = cursor.fetchall()
    return str(result)
                   
run(app, host='0.0.0.0',port=5000,debug=True)

In [2]:
import sqlite3
from bottle import Bottle, run, template

app = Bottle()

@app.route('/todo')
@app.route('/')
def todo_list():
    connection = sqlite3.connect('todo.db')
    cursor = connection.cursor()
    cursor.execute("SELECT id, task FROM todo WHERE status LIKE '1'")
    result = cursor.fetchall()
    output = template('make_table',rows=result)
    return output

run(app, host='0.0.0.0',port=5000,debug=True)

Bottle v0.12.16 server starting up (using WSGIRefServer())...
Listening on http://0.0.0.0:5000/
Hit Ctrl-C to quit.

10.0.2.2 - - [21/Mar/2019 10:35:16] "GET / HTTP/1.1" 200 672
/home/vagrant/.local/share/virtualenvs/Jupyternotebooks-vMpxa5dg/lib/python3.6/site-packages/bottle.py:3131: ResourceWarning: unclosed <socket.socket fd=60, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('0.0.0.0', 5000)>
  pass


In [5]:
import sqlite3
from bottle import Bottle, run, template, request

app = Bottle()

@app.route('/todo')
@app.route('/')
def todo_list():
    connection = sqlite3.connect('todo.db')
    cursor = connection.cursor()
    cursor.execute("SELECT id, task FROM todo WHERE status LIKE '1'")
    result = cursor.fetchall()
    output = template('make_table',rows=result)
    return output

@app.route('/new',method='GET')
def new_task():
    return template('new_task.tpl')

@app.route('/new',method='POST')
def new_item_do():
    
    new_item = request.POST.task.strip()
    
    connection = sqlite3.connect('todo.db')
    cursor = connection.cursor()
    
    cursor.execute("INSERT INTO todo (task,status) VALUES (?,?)",(new_item,0))
    new_id = cursor.lastrowid
    
    connection.commit()
    cursor.close()
    
    return f"""
    <meta HTTP.EQUIV="REFRESH" content="5; url={request.urlparts[0]+'://'+request.urlparts[1]}/">
    <p>The new task was iserted into the database, the ID is { new_id }</p>"""

run(app, host='0.0.0.0',port=5000,debug=True)

Bottle v0.12.16 server starting up (using WSGIRefServer())...
Listening on http://0.0.0.0:5000/
Hit Ctrl-C to quit.

10.0.2.2 - - [21/Mar/2019 11:07:45] "GET /new HTTP/1.1" 200 485
10.0.2.2 - - [21/Mar/2019 11:07:48] "GET /new HTTP/1.1" 200 485
/home/vagrant/.local/share/virtualenvs/Jupyternotebooks-vMpxa5dg/lib/python3.6/site-packages/bottle.py:3131: ResourceWarning: unclosed <socket.socket fd=60, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('0.0.0.0', 5000)>
  pass
